<a href="https://colab.research.google.com/github/petruzzellialessandro/RCSmovie/blob/main/SequenceModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-g3m5dkcc
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-g3m5dkcc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 53.8 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5664847 sha256=250bf1b9be2ffb36d898d689ad47eb91a173fd8252a185ff4ff3720b2a3fb454
  Stored in directory: /tmp/pip-ephem-wheel-cache-85jr1jzs/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
tokenizers                    0.13.2
transformers                  4.25.0.dev0


# BERT Per la predizione e completamento delle sequenze

## Addestramento del modello

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
path = r"/content/drive/MyDrive/Colab Notebooks/test.csv"
x = pd.read_csv(path)
new_x = []
for v in x.values:
  new_x.append(v[0])
path_eval = r"/content/drive/MyDrive/Colab Notebooks/eval.csv"
x_eval = pd.read_csv(path_eval)

Tokenizer delle sequenze e aggiunta dei token al tokenizer pre-addestrato di Bert

In [ ]:
from transformers import BertTokenizer

tokenizer_path = '/content/drive/MyDrive/Colab Notebooks/tokenizer_v2.bin'
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", add_special_token=['[POS]', '[NEG]', '[pos]', '[neg]'], do_lower_case=False, min_frequency = 0)
#tokenizer.from_pretrained("bert-base-uncased")
for temp in new_x:
  #tokenizer.add_tokens(temp.replace("[POS]","").replace("[NEG]","").split(" "))
  tokenizer.add_tokens(temp.split(" "))
"""tokenizer.add_tokens('[POS]')
tokenizer.add_tokens('[NEG]')"""
tokenizer.save_pretrained(tokenizer_path)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

('/content/drive/MyDrive/Colab Notebooks/tokenizer_v2.bin/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/tokenizer_v2.bin/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/tokenizer_v2.bin/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/tokenizer_v2.bin/added_tokens.json')

Effettiva tokenizzazione delle sequenze di training

In [ ]:
def tokenize(text):
  return tokenizer.encode(text, truncation=False, padding="max_length", max_length=46, return_special_tokens_mask=True)
train = x['0'].map(tokenize)
eval = x_eval['0'].map(tokenize)

In [ ]:
train, eval

(0        [101, 30522, 30523, 30524, 30525, 30526, 30527...
 1        [101, 30534, 30535, 30536, 30537, 30538, 102, ...
 2        [101, 30529, 30539, 30540, 30536, 30541, 30542...
 3        [101, 30555, 30556, 30557, 30558, 30559, 30560...
 4        [101, 30568, 30564, 30569, 30570, 30571, 30572...
                                ...                        
 99995    [101, 30682, 30805, 30590, 30573, 30627, 30731...
 99996    [101, 30664, 30636, 30721, 30735, 30890, 30913...
 99997    [101, 30646, 30879, 30886, 30690, 30839, 30821...
 99998    [101, 30857, 30575, 30688, 30881, 30888, 30787...
 99999    [101, 30830, 30629, 30675, 30881, 30777, 30888...
 Name: 0, Length: 100000, dtype: object,
 0     [101, 30637, 30664, 30744, 30615, 30745, 30886...
 1     [101, 30785, 30833, 30706, 30602, 30760, 30552...
 2     [101, 30538, 30700, 30854, 30539, 30762, 30555...
 3     [101, 30631, 30614, 30655, 30654, 30897, 30776...
 4     [101, 30688, 30553, 30557, 30871, 30635, 30906...
              

Config del modello. Gestire i batch per far accelerare la costruzione del modello. Va rivista anche la strategia di ricerca perché Adam presto sarà deprecata.

In [ ]:
from transformers import BertForMaskedLM, BertConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling

model_config = BertConfig(vocab_size=len(tokenizer.get_vocab()), max_position_embeddings=46)
model = BertForMaskedLM(config=model_config)

training_args = TrainingArguments(
    optim='adamw_torch',
    output_dir="/content/drive/MyDrive/Colab Notebooks/model_v2.bin",          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=64, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=128,  # evaluation batch size
    logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

Con `DataCollatorForLanguageModeling` decidiamo mascherare i dati. In questo caso c'è il 20% di probabilità che una sequenza abbia una maschera all'interno.

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train,
    eval_dataset=eval #Serve il dataset di valutazione
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 100000
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 8
  Total optimization steps = 1950
  Number of trainable parameters = 109467855


Step,Training Loss,Validation Loss
1000,5.735900,5.456806


***** Running Evaluation *****
  Num examples = 100
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/model_v2.bin/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/model_v2.bin/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/model_v2.bin/checkpoint-1000/pytorch_model.bin


Step,Training Loss,Validation Loss
1000,5.735900,5.456806




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1950, training_loss=5.584011418269231, metrics={'train_runtime': 9575.8752, 'train_samples_per_second': 104.429, 'train_steps_per_second': 0.204, 'total_flos': 2.36436355929312e+16, 'train_loss': 5.584011418269231, 'epoch': 10.0})

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/model_v2.bin"
model.save_pretrained(model_path)

Configuration saved in /content/drive/MyDrive/Colab Notebooks/model_v2.bin/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/model_v2.bin/pytorch_model.bin


## Lettura del modello

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import BertForMaskedLM

model_path = "/content/drive/MyDrive/Colab Notebooks/model_v2.bin"
tokenizer_path = '/content/drive/MyDrive/Colab Notebooks/tokenizer_v2.bin'
model = BertForMaskedLM.from_pretrained(model_path)

loading configuration file /content/drive/MyDrive/Colab Notebooks/model_v2.bin/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 46,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30927
}

loading weights file /content/drive/MyDrive/Colab Notebooks/model_v2.bin/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/model

In [ ]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer_path, top_k=100)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding [NEG]E91 to the vocabulary
Adding [NEG]E85 to the vocabulary
Adding [NEG]I63 to the vocabulary
Adding [NEG]I31 to the vocabulary
Adding [POS]I24 to the vocabulary
Adding [POS]I14 to the vocabulary
Adding [POS]E46 to the vocabulary
Adding [NEG]E93 to the vocabulary
Adding [NEG]I4 to the vocabulary
Adding [POS]E95 to the vocabulary
Adding [POS]I8 to the vocabulary
Adding  to the vocabulary
Adding [NEG]E37 to the vocabulary
Adding [NEG]I9 to the vocabulary
Adding [NEG]E32 to the vocabulary
Adding [POS]I7 to the vocabulary
Adding [POS]I35 to the vocabulary
Adding [NEG]E77 to the vocabulary
Adding [POS]I65 to the vocabulary
Adding [NEG]I8 to the vocabulary
Adding [POS]E35 to the vocabulary
Adding [NEG]I12 to the vocabulary
Adding [NEG]I47 to the vocabulary
Adding [POS]I48 to the vocabulary
Adding [NEG]E62 to the vocabulary
Adding [P

In [ ]:
x_eval['0'][0]

'[POS]I31 [POS]E94 [POS]E75 [NEG]E3 [POS]E29 [POS]E91 [NEG]E57 [NEG]E97 '

In [ ]:
for complete_seg in x_eval['0']:
  masked = complete_seg+"[MASK]"
  pos = False
  neg = False
  for resp in fill_mask(masked):
    if '[POS]I' in resp['token_str'] and not pos:
      print(resp)
      pos = True
    if '[NEG]I' in resp['token_str'] and not neg:
      print(resp)
      neg = True
    if pos and neg:
      break

{'score': 0.002697339514270425, 'token': 30622, 'token_str': '[NEG]I97', 'sequence': '[POS]I31 [POS]E94 [POS]E75 [NEG]E3 [POS]E29 [POS]E91 [NEG]E57 [NEG]E97 [NEG]I97'}
{'score': 0.0026886381674557924, 'token': 30585, 'token_str': '[POS]I49', 'sequence': '[POS]I31 [POS]E94 [POS]E75 [NEG]E3 [POS]E29 [POS]E91 [NEG]E57 [NEG]E97 [POS]I49'}
{'score': 0.0027387996669858694, 'token': 30585, 'token_str': '[POS]I49', 'sequence': '[NEG]I35 [POS]E97 [NEG]E11 [NEG]E45 [NEG]I5 [NEG]E48 [POS]I80 [NEG]E90 [POS]E84 [POS]I10 [POS]I49'}
{'score': 0.002698419848456979, 'token': 30850, 'token_str': '[NEG]I36', 'sequence': '[NEG]I35 [POS]E97 [NEG]E11 [NEG]E45 [NEG]I5 [NEG]E48 [POS]I80 [NEG]E90 [POS]E84 [POS]I10 [NEG]I36'}
{'score': 0.002758779563009739, 'token': 30611, 'token_str': '[POS]I41', 'sequence': '[POS]I35 [NEG]I20 [NEG]I26 [NEG]E77 [POS]E20 [NEG]I34 [POS]I98 [POS]E4 [NEG]I34 [POS]E72 [NEG]E83 [NEG]I82 [POS]I41'}
{'score': 0.0026562735438346863, 'token': 30622, 'token_str': '[NEG]I97', 'sequence': 